In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
df = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/train.csv')


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df.info()

In [ ]:
for i in df.columns:
    df[i] = df[i].astype('int16')

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
df.drop(['Id','Soil_Type15','Soil_Type7'],axis = 1,inplace = True)

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
li = [x for x in df.columns]

In [ ]:
df.info()

In [ ]:
df.tail()

In [ ]:
#df['ecldn_dist'] = (df['Horizontal_Distance_To_Hydrology']**2 + df['Vertical_Distance_To_Hydrology']**2)**0.5

In [ ]:
#df.head()

In [ ]:
for cols in df.columns:
    print(cols,':',df[cols].value_counts().nunique())

In [ ]:
df['Cover_Type'].value_counts()

In [ ]:
import numpy as np
df.drop(df.loc[df['Cover_Type'] == 5].index, inplace = True)

In [ ]:
df.drop(df.loc[df['Cover_Type'] == 4].index, inplace = True)

In [ ]:
df.loc[df['Cover_Type'] == 4].index

In [ ]:
df.loc[df['Cover_Type'] == 4].index

In [ ]:
len(df)

In [ ]:
x = df.drop('Cover_Type', axis = 1)
y = df[['Cover_Type']]

In [ ]:
num_cols = [
    "Elevation",
    "Aspect",
    "Slope",
    "Horizontal_Distance_To_Hydrology",
    "Vertical_Distance_To_Hydrology",
     "Horizontal_Distance_To_Roadways",
    "Hillshade_9am",
    "Hillshade_Noon",
    "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points",
]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler =  StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])


In [ ]:
df

In [ ]:
df.shape

# Validation

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size = 0.70,random_state = 456)
#x_train = df.drop('Cover_Type',axis = 1)
#y_train = df['Cover_Type']

In [ ]:
x_test.shape

# RandomSearchCV

In [ ]:
from xgboost import XGBClassifier as xgb
xg = xgb(tree_method = 'gpu_hist')
xg

In [ ]:
params = {'gamma':[0.1,0.2,0.3,0.4,0.5,0.6,0.7],
         'learning_rate':[0.01,0.05,0.08,0.1,0.2,0.3,0.4,0.5],
         'n_estimators':[80,100,120,140,160,180,200],
         'max_depth':[2,3]}

# XGBClassifier

In [ ]:
xgb_params = {
    'objective': 'multi:softmax',
    'eval_metric': 'merror', 
    
    
    'max_depth': 7
    }
xg = xgb(n_estimators = 180, learning_rate= 0.5, gamma= 0.1,**xgb_params).fit(x_train,y_train)

In [ ]:
predXGB = xg.predict(x_test)

In [ ]:
xg.score(x_train,y_train)

In [ ]:
xg.score(x_test,y_test)

# Test Data

In [ ]:
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/test.csv')

In [ ]:
df_test.info()

In [ ]:
for t in df_test.columns:
    df_test[t] = df_test[t].astype('int16')

In [ ]:
df_test.info()

In [ ]:
df_test.drop(['Id'],axis = 1,inplace = True)

In [ ]:
df_test.drop(['Soil_Type7','Soil_Type15'],axis = 1, inplace = True)

In [ ]:
#df_test['ecldn_dist'] = ((df_test['Horizontal_Distance_To_Hydrology'])**2 + (df_test['Vertical_Distance_To_Hydrology'])**2)**0.5

In [ ]:
df_test.head()

In [ ]:
y_test = df_test

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df_test.isnull().sum()

In [ ]:
#df_test.drop('ecldn_dist',axis = 1, inplace = True)

In [ ]:
predXGB = xg.predict(y_test)
predXGB

In [ ]:
submission = pd.DataFrame({'Id' : [x for x in range(4000000,5000000)]})

In [ ]:
submission['Cover_Type'] = predXGB

In [ ]:
submission

In [ ]:
submission['Cover_Type'].value_counts()

In [ ]:
submission.to_csv('submission.csv',index = False)